# QC2 Data

*Lucas Visscher, Vrije Universiteit Amsterdam, 2023*

All data required for orbital manipulation is to be stored on the file orbitals.h5

This file is in hdf5 format allowing for easy import and export of the data from/to other formats. 
The structure is given by the data schema contained in the file `QC2schema.txt`. This file is processed by a number of Python scripts and then defines the hdf5 directory structure. We will illustrate the extraction of data into Python by means of the included `scf_hf.h5` file that was generated by the DIRAC program and serves there as an example for assessing DIRAC-generated data. The python scripts that process the schema are identical to the ones used in DIRAC and should be kept in sync as much as possible. The DIRAC schema itself is necessarily different as more information is stored and is given in the file `DIRAC23schema.txt`.

Dependencies for this notebook are:
- process_schema.py: a collection of python functions to process the schema and read/write hdf5 files.
- scf_hf.h5 : the above-mentioned sample data set from DIRAC, resulting from a calculation on the HF molecule.
- QC2schema.txt and DIRAC23schema.txt: the definitions of the data schema's

Please copy these file into the directory in which you start this Jupyter Notebook before running the next code cell and make sure that you have also installed the h5py library (`conda install h5py` if you are using the conda package manager) in your python environment.

In [ ]:
from process_schema import read_hdf5, write_hdf5
data = read_hdf5('scf_hf.h5')

The above gave a list of all data contained on the file in which the hierarchical structure of the labels is clearly visible. Let's consider printing out the angular momenta of each basis function in the AO basis. This is done below:

In [ ]:
# Print the large component basis
orbmom = data['/input/aobasis/1/orbmom']['value']
for shell, lvalue  in enumerate(orbmom):
    print ('Shell {} has l-value {}'.format(shell+1,lvalue-1))

Note that Python numbers from zero so we add one to the value of the shell counter to be consistent with how this is also done internally in DIRAC (a Fortran program). On the other hand we see that in DIRAC an orbmom value of 1 indicates an s-function. We therefore need to subtract one from the orbmom numbers to obtain the actual l-value.

In the following cell we will list the AO basis together with its expansion centers, exponential parameters and contraction coefficients, after first defining the proper symbols (s, p, d, f, g) to indicate the functions. We also use numpy to reshape the data for ease of printing. In DIRAC all arrays are always stored as 1-dimensional while they are usually conceptually multidimensional.

In [ ]:
import numpy as np

# Get the data for the AO basis into numpy arrays for convenience.
n_shells = data['/input/aobasis/1/n_shells']['value'][0]
orbmom = data['/input/aobasis/1/orbmom']['value']
center = np.array(data['/input/aobasis/1/center']['value']).reshape((n_shells,3))
exponents = np.array(data['/input/aobasis/1/exponents']['value'])
contractions = np.array(data['/input/aobasis/1/contractions']['value'])

# Detailed printing of the AO basis. 
name_l = ['-','s','p','d','f','g']
print ('     Center           Type Exponent Coefficient\n',46*'-')
i_exp = 0
i_con = 0
for shell, lvalue  in enumerate(orbmom):
    n_cont = data['/input/aobasis/1/n_cont']['value'][shell]
    if data['/input/aobasis/1/n_cont']['value'][shell] > 1: 
        print('WARNING: can not print generally contracted AO sets')
    n_prim = data['/input/aobasis/1/n_prim']['value'][shell]
    print ('{:6.3f} {:6.3f} {:6.3f}    {}  {:8.2e} {:8.4f}'.format(*center[shell,0:3],name_l[lvalue],
                                            exponents[i_exp],contractions[i_con]))
    i_exp += 1
    i_con += 1
    for prim in range(n_prim-1):
        print ('{} {:8.2e} {:8.4f}'.format(26*' ',exponents[i_exp],contractions[i_con]))
        i_exp += 1
        i_con += 1
    i_con += (n_cont-1)*n_prim # This should be OK for generally contracted sets but is not checked !

Note that each shell has its own center defined, this is done for simplicity and ease of resorting basis sets. Note also that the coordinates are slightly different from the input values as DIRAC will by default reorient the molecule such that the center of mass is in the origin. This is also visible in the coordinates of the two atoms of this molecule that are updated and can be extracted also from the input section. Note that these are stored in Ångstrom, while the expansion center coordinates are stored in Bohrs (atomic units).

In [ ]:
# Print the molecular coordinates
print (data['/input/molecule/geometry']['value'])

Turning to the results section we will print first the orbital energies obtained in the SCF. These are stored in atomic units (Hartrees in this case) as well.

In [ ]:
# Print the orbital energies
print (data['/result/wavefunctions/scf/mobasis/eigenvalues']['value'])

Other data can be extracted in a similar way, please consult file `DIRACschema.txt` for concise definitions of all the data that is stored.

## Conversion from DIRAC to QC2 data

In order to convert these DIRAC data to our own QC2 format, we need first to read our data schema and create a valid dictionary in which only all `'value'` items are missing. This is done below, after also writing the generated labels to a text file that is more easily processed from a Fortran program than the original schema file.

In [ ]:
from process_schema import read_schema, write_schema

qc2_schema, qc2_flatschema = read_schema('QC2schema.txt')
write_schema('QC2labels.txt',qc2_flatschema)
qc2_data = qc2_flatschema.copy()

DIRAC has all of the data needed for QC2 but some data is named differently, and the optional descriptors are not defined. We set these first manually and then adjust for the difference in directories used for storing the MO basis. The data is subsequently written to an hdf5 file and we are ready.

In [ ]:
qc2_data['/input/aobasis/1/descriptor']['value']  = 'DIRAC'
qc2_data['/result/mobasis/1/descriptor']['value'] = 'DIRAC_scf'
for label, item in qc2_data.items():
    if 'mobasis' in label:
        diraclabel = label.replace('mobasis/1','wavefunctions/scf/mobasis')
    else:
        diraclabel = label
    if item['use'] == 'required' and item['type'] != 'composite': 
        try:
            qc2_data[label]['value'] = data[diraclabel]['value']
        except:
            print('required data {} not found'.format(label))

write_hdf5('orbitals.h5',qc2_data)
    